In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import traceback


In [117]:
def hanabank_crawling(url):

    # 웹 페이지에 접근하여 HTML 데이터 가져오기
    response = requests.get(url)
    html = response.text
    
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(html, 'html.parser')
    main = soup.find('h3', class_='main-title')
    if main:
        main_title = main.text.strip()
        explanation = main.find('span', class_='sub')
        main_title = main_title.split("\r")[0]
        if explanation:
            explanation = explanation.text.strip()

    col_data = soup.find('dl', class_='prodcutInfo')
    col_titles = [i.text.replace("\n", "").replace("\r", "").replace("\t", "").replace(" ", "") for i in col_data.find_all('dt')]
    col_values = [i.text for i in col_data.find_all('dd')]
    
    df_list = list()
    for i in range(len(col_titles)):
        df_list.append([col_titles[i].strip(), col_values[i]])
        
    df_list.append(["상품명", main_title])
    df_list.append(["상품설명", explanation])
    df = pd.DataFrame(df_list, columns=["구분", "내용"])
    df = df.drop_duplicates(["구분"])
    
    # null인 행들을 순회하면서 이전 행의 '구분'에 추가하고 해당 행 삭제
    null_rows = df[(df["구분"].isin(["", "None"])) | (df["구분"].isna())]
    
    for index, row in null_rows.iterrows():
        prev_index = index - 1
        if prev_index >= 0:
            while True:
                try:
                    df.at[prev_index, "내용"] += f"\n{row['내용']}"
                    break
                except KeyError:
                    prev_index -= 1
                    continue
        df.drop(index, inplace=True)
    # df.reset_index(drop=True, inplace=True)   
    df.set_index(["구분"], inplace=True) 
    try:
        rate = col_data.find('a', class_='btnXsmall')['href']
        df.at["금리", "내용"] = rate
    except Exception:
        pass
    return df.T.reset_index(drop=True)

In [118]:
url = "https://www.kebhana.com/cont/mall/mall08/mall0801/mall080103/1495937_115188.jsp"
etc = hanabank_crawling(url)
etc

구분,상품특징,가입대상,예금과목,가입방법,상품명,전환여부,금리,우대서비스,연계·제휴서비스,혁신금융서비스승인에관한사항,세제혜택,유의사항,원금및이자지급제한,자료열람요구권,민원처리및분쟁조정절차,예금자보호,상품설명
0,제휴사의 특정서비스 등록 및 필수요건 충족시 우대 서비스를 제공하는 통장,쿠팡페이㈜의 연계·제휴서비스를 이용하는 법인 및 개인사업자(1인 1계좌만 가능)\n,\n기업자유예금\n,\n쿠팡페이㈜의 웹 또는 앱페이지에 연계된 『하나은행 비대면계좌개설서비스』를 통해서...,쿠팡 셀러 통장,불가 (다른 예금에서 이 예금으로 전환 또는 이 예금에서 다른 예금으로 전환은 불가),/app/portal/mkt/contents/rate_p02_01.do?action...,이 예금을 통해 필수요건 충족시 우대서비스 제공 \r\n 【필수요건】 ...,\n\n\n연계·제휴 서비스\n\n\n\n\n\n\n구 분\n내 ...,\n\n\n혁신금융서비스 승인에 관한사항\n\n\n구 분\n\n\n\n\n\n\...,\n비과세 종합저축으로 가입 가능 (전 금융기관 통합한도 범위내)\n관련 세법이 ...,\n\n우대서비스 제공은 ‘필수요건 충족시’ 에만 적용됩니다.\n우대서비스 내용 및...,"\n\n계좌에 압류, 가압류, 질권설정 등이 등록될 경우 원금 및 이자지급 제한될 ...",\n\n금융소비자는 분쟁조정 또는 소송의 수행 등 권리구제를 위한 목적으로 은행이 ...,"\r\n 본 상품에 대한 문의사항 또는 민원 상담이 필요하시면 영업점, 고객센...","\n\n이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호 한도는 본 은행...",쿠팡페이 셀러월렛서비스를 이용 시 우대 서비스를 제공하는 통장


In [119]:
path = "../data/은행상품/하나은행/하나은행_상품_URL.txt"
df = pd.DataFrame()
error_log_path = "./error.txt"
error_f = open(error_log_path, "w", encoding="utf-8-sig")

with open(path, "r", encoding="utf-8-sig") as f:
    while True:
        url = f.readline().strip()
        if not url:
            break
        try:
            etc = hanabank_crawling(url)
            df = pd.concat([df, etc], axis=0)
            error_f.write(url+"\n")
            error_f.write(", ".join(etc.columns)+"\n")
        except Exception as e:
            error_f.write(url+"\n")
            traceback.print_exc(file=error_f)  # traceback를 파일에 직접 기록
            pass
error_f.close()

# 변경하고자 하는 특정 컬럼의 이름 설정
target_column = ["상품명", "상품설명"]

# 특정 컬럼의 위치를 변경
columns_except_target = [col for col in df.columns if col not in target_column]
target_column.extend(columns_except_target)
df = df[target_column]
df.reset_index(drop=True, inplace=True)

In [157]:
path = "../data/은행상품/하나_backup.csv"
df.to_csv(path, encoding="utf-8-sig")

In [159]:
path = "../data/은행상품/하나_backup.csv"
df = pd.read_csv(path, encoding="utf-8-sig", index_col=0)
sorted(df.columns)


['가입금액',
 '가입기간',
 '가입기간중분할해지',
 '가입기간중중도해지수수료율',
 '가입대상',
 '가입및적립금액',
 '가입방법',
 '가입상품의종류',
 '갱신',
 '건별입금가능액',
 '결정지수',
 '계약의해지',
 '금리',
 '기준지수',
 '기타',
 '납입금액',
 '납입한도',
 '다자녀가구특별우대금리',
 '만기금리',
 '만기수취이자예시',
 '만기앞당김지급',
 '만기해지및후원방식',
 '만기후금리',
 '매매대금의흐름',
 '매칭지원금',
 '모집한도',
 '민원처리및분쟁조정절차',
 '발행형식',
 '부가서비스',
 '분할인출',
 '분할해지',
 '비과세',
 '상품간전환',
 '상품내용변경에관한사항',
 '상품명',
 '상품명지정',
 '상품설명',
 '상품유형',
 '상품종류',
 '상품특징',
 '세제혜택',
 '소득공제',
 '수수료면제',
 '수수료우대',
 '수익구조',
 '양도및담보제공',
 '양수도',
 '업무방법과이용방법',
 '연계·제휴서비스',
 '연계·제휴서비스변경내용사전통지방법',
 '연계·제휴서비스이행책임',
 '연계·제휴서비스제공기간',
 '연계·제휴서비스제공요건',
 '예금과목',
 '예금담보대출',
 '예금신규일',
 '예금자보호',
 '예금종류',
 '예치가능',
 '예치통화',
 '용도',
 '우대금리',
 '우대금리(쿠폰)',
 '우대서비스',
 '원금및이자지급제한',
 '위법계약해지권',
 '유의사항',
 '은행계좌계정과목',
 '이용제한사항',
 '이자계산및이자지급방법',
 '이자지급방법',
 '이자지급방식',
 '이자지원금',
 '일부해지',
 '입금가능통화',
 '자료열람요구권',
 '재가입',
 '재예치',
 '저축기간',
 '적립금액',
 '적립방법',
 '적립중지',
 '적립한도',
 '적용금리',
 '전환신규',
 '전환여부',
 '정부기여금',
 '정부지원금',
 '준비서류',
 '중도해지',
 '중도해지금리',
 '증권계좌이용안내',
 '직장인응원서비스',
 '청약기간'

In [156]:
# df['특별금리'] = df['청년응원특별금리'].fillna('') + df['HAPPYYEAR특별금리(2021.09.01)'].fillna('')
# # df.drop(['청년응원특별금리', 'HAPPYYEAR특별금리(2021.09.01)'], axis=1, inplace=True)

# df['금리'] = df['금리'].fillna('') + df['기본금리'].fillna('')
# df.drop(['기본금리'], axis=1, inplace=True)

# df['만기자동재예치선택시'] = df['만기자동재예치선택시'].fillna('') + df['만기자동재예치선택시(총3회)'].fillna('')
# df['만기자동재예치선택시'] = df['만기자동재예치선택시'].fillna('') + df['만기자동재예치선택시(총5회)'].fillna('')
# df.drop(['만기자동재예치선택시(총3회)', '만기자동재예치선택시(총5회)'], axis=1, inplace=True)

# df['기타'] = df['기타'].fillna('') + df['기타사항'].fillna('')
# df.drop(['기타사항'], axis=1, inplace=True)

# df['재예치'] = df['재예치'].fillna('') + df['만기자동재예치선택시'].fillna('')
# df['재예치'] = df['재예치'].fillna('') + df['재예치선택시'].fillna('')
# df.drop(['만기자동재예치선택시', '재예치선택시'], axis=1, inplace=True)

# df['우대서비스'] = df['우대서비스'].fillna('') + df['우대서비스대상'].fillna('')
# df.drop(['우대서비스대상'], axis=1, inplace=True)

# df['예치통화'] = df['예치통화'].fillna('') + df['예치가능통화'].fillna('')
# df.drop(['예치가능통화'], axis=1, inplace=True)

In [160]:
# filtered_df = df[[col for col in df.columns if col in ["우대금리", "우대금리(쿠폰)"]]]
filtered_df = df[[col for col in df.columns if "가입기간" in col]]
rows_with_values = filtered_df[filtered_df.apply(lambda row: row.count() >= 2, axis=1)]
print(len(rows_with_values))
filtered_df

2


,가입기간,가입기간중중도해지수수료율,가입기간중분할해지
0,1년제,NaN,NaN
1,"1개월 연동형 : 1년제3, 6개월 연동형 : 1, 2, 3, 5년제1년 연동형 :...",NaN,NaN
2,1개월~5년(일 단위 만기 지정 가능),NaN,NaN
3,1개월 이상 ~ 5년 이내 일단위,NaN,NaN
4,최장 31년까지 가능거치기간 : 1년(선택사항)원리금지급기간 : 1년~30년(연단위...,NaN,NaN
...,...,...,...
78,NaN,NaN,NaN
79,제한없음,NaN,NaN
80,제한없음,NaN,NaN
81,2023.12.19(화) ~ 2024.06.19(수) [6개월],\n가입기간중 중도해지 수수료 : 해지원금 x 예치기간별 중도해지 수수료율\n\n\...,가능(만기해지 제외 2회만 가능)\n※ 분할해지금액에 대해 ‘가입기간중 중도해지수수...


In [155]:
rows_with_values

,재예치,예치가능,예치통화,예치가능통화
67,이 적금의 신규개설 시 또는 만기일 이전에 손님이 적금의 재예치를 신청한 때에는 만...,NaN,미국 달러화 (USD),NaN
